# scSpatial Seurat for StarMap

In [ ]:
# install.packages("Seurat")

## Library import

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(magrittr)

library(viridis)

set.seed(1)

# Multicore CPU usage activation - Future
library(future)

plan("multicore", workers = 10) 
# Set RAM Size to 3/4 of total RAM
options(future.globals.maxSize = 16000 * 1024^2)

future.seed=TRUE # Removes future-generated statistical errors

## Load data

In [ ]:
library(data.table)

cell_by_gene <- fread("cell_by_gene.csv", data.table = FALSE)
cell_metadata <- fread("cell_metadata.csv", data.table = FALSE)

# Extract cell barcodes and gene names
cell_barcodes <- cell_by_gene[, 1]  # First column contains cell barcodes
gene_names <- colnames(cell_by_gene)[-1]  # Gene names are column names

# Ensure unique gene names
gene_names <- make.unique(gene_names)  

# Transpose matrix
cell_by_gene <- t(cell_by_gene[, -1])  # Remove first column (cell barcodes) before transposing

# Assign correct row & col names
rownames(cell_by_gene) <- gene_names  # Genes as row names
colnames(cell_by_gene) <- make.unique(cell_barcodes)  # Ensure unique cell names

# Convert to matrix
cell_by_gene <- as.matrix(cell_by_gene)

# Create Seurat object
Seurat <- CreateSeuratObject(counts = cell_by_gene)

# Fix metadata cell names
cell_metadata$V1 <- gsub("_\\d+$", "", cell_metadata$V1)  # Remove suffixes like "_10" if needed
rownames(cell_metadata) <- make.unique(cell_metadata$V1)  # Ensure unique rownames

# Remove first column from metadata
cell_metadata <- cell_metadata[, -1]

# Add metadata
Seurat <- AddMetaData(Seurat, metadata = cell_metadata)

Seurat

In [ ]:
VlnPlot(Seurat, features = "nFeature_RNA")
VlnPlot(Seurat, features = "nCount_RNA")

In [ ]:
Seurat <- NormalizeData(Seurat, normalization.method = "LogNormalize", scale.factor = 10000)
all.genes <- rownames(Seurat)
Seurat <- ScaleData(Seurat, features = all.genes)

Seurat

In [ ]:
distance_clusters <- read.csv("cluster_info.csv")
Seurat <- AddMetaData(Seurat, metadata = distance_clusters$Cluster, col.name = "CellType")
Seurat <- SetIdent(Seurat, value = "CellType")

Seurat

In [ ]:
head(Seurat@meta.data)

## Spatial Cluster Plot

In [ ]:
Seurat@meta.data$center_x <- as.numeric(Seurat@meta.data$center_x)
Seurat@meta.data$center_y <- as.numeric(Seurat@meta.data$center_y)

# Plot clusters with black background
ggplot(Seurat@meta.data, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 1) +
  scale_color_manual(values = rainbow(length(unique(Seurat$CellType)))) + # Custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Clusters", color = "CellType")

ggsave(
  "Spatial Cluster Plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 42,
  height = 40,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

## Spatial Feaute Plot

In [ ]:
# Define the genes you want to plot
genes <- c("TNFRSF1B", "IGKC", "TRBC1", "COL3A1")  # Replace with actual gene names

# Ensure spatial coordinates are numeric
Seurat@meta.data$center_x <- as.numeric(Seurat@meta.data$center_x)
Seurat@meta.data$center_y <- as.numeric(Seurat@meta.data$center_y)

# Convert Seurat object to a data frame for plotting
expression_data <- Seurat@meta.data
for (gene in genes) {
  expression_data[[gene]] <- as.numeric(FetchData(Seurat, vars = gene)[,1])  # Ensure numeric
}

# Generate individual gene expression plots
plot_list <- lapply(genes, function(gene) {
  ggplot(expression_data, aes(x = center_x, y = center_y, color = !!sym(gene))) +
    geom_point(size = 0.25) +
    scale_color_viridis_c(option = "viridis") +  # Use a continuous color scale
    theme_void() +
    theme(
      panel.background = element_rect(fill = "black", color = "black"), 
      plot.background = element_rect(fill = "black"),
      legend.text = element_text(color = "white"),
      legend.title = element_text(color = "white"),
      plot.title = element_text(family = "Helvetica", face = "bold", color = "white", hjust = 0.5, size = 18), # Title settings
      plot.margin = margin(0, 0, 0, 0)  # Remove extra margins
    ) +
    labs(title = gene, color = "")  # Title = Gene name
})

# Arrange plots in a 2x2 grid and remove the outer border
final_plot <- wrap_plots(plot_list, ncol = 2) +
  plot_annotation(theme = theme(plot.background = element_rect(fill = "black", color = NA)))  # Remove outer white border

ggsave( 
  "Spatial Feaute Plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 44,
  height = 40,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

## Spatial Subcluster Plot

In [ ]:
# Convert spatial coordinates to numeric
Seurat@meta.data$center_x <- as.numeric(Seurat@meta.data$center_x)
Seurat@meta.data$center_y <- as.numeric(Seurat@meta.data$center_y)

# Filter for clusters 29, 3, and 6
subset_meta <- Seurat@meta.data[Seurat@meta.data$CellType %in% c(29, 20, 15), ]

# Define custom colors for selected clusters
custom_colors <- c("29" = "purple", "20" = "green", "15" = "yellow")  # Customize as needed

# Plot clusters with black background
ggplot(subset_meta, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 1) +
  scale_color_manual(values = custom_colors) + # Use custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Selected Clusters", color = "Cluster")

ggsave(
  "Spatial SubCluster Plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 42,
  height = 40,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

## FOV selection

In [ ]:
# Define FOV boundaries
x_min <- 3000
x_max <- 8000
y_min <- 7500
y_max <- 12500

# Filter cells within the FOV
subset_cells <- rownames(Seurat@meta.data[
  Seurat@meta.data$center_x >= x_min & 
  Seurat@meta.data$center_x <= x_max & 
  Seurat@meta.data$center_y >= y_min & 
  Seurat@meta.data$center_y <= y_max, 
])

# Subset Seurat object
Seurat_FOV <- subset(Seurat, cells = subset_cells)

# Check dimensions of the new object
print(dim(Seurat_FOV))

## FOV Spatial Cluster Plot

In [ ]:
Seurat_FOV@meta.data$center_x <- as.numeric(Seurat_FOV@meta.data$center_x)
Seurat_FOV@meta.data$center_y <- as.numeric(Seurat_FOV@meta.data$center_y)

# Plot clusters with black background
ggplot(Seurat_FOV@meta.data, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 2) +
  scale_color_manual(values = rainbow(length(unique(Seurat_FOV$CellType)))) + # Custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Clusters", color = "CellType")

ggsave(
  "Spatial FOV Cluster Plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 42,
  height = 40,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

## Rename Clusters 

In [ ]:
levels(Seurat) <- c('1','2','3','4','5','6','7','8','9','10',
                    '11','12','13','14','15','16','17','18','19','20',
                   '21','22','23','24','25','26','27','28','29','30','31')

new.cluster.ids <- c('1','2','3','4','5','6','7','8','9','10',
                    '11','12','13','14','15','16','17','18','19','20',
                   '21','22','23','24','25','26','27','28','29','30','31')

names(new.cluster.ids) <- levels(Seurat)

Seurat <- RenameIdents(Seurat, new.cluster.ids)

## Perform DGE

In [ ]:
deg0 <- FindMarkers(Seurat, ident.1 = '29',ident.2 = '1', only.pos = FALSE, logfc.threshold = 1., min.pct = 0)
deg0 <- deg0[order(deg0$avg_log2FC,decreasing=TRUE),]
head(deg0)

## Dot plot of the distance cluster genes

In [ ]:
cluster_features = c('SLC35E2B', 'TNFRSF25', 'H6PD',
                     'ARHGEF10L', 'KPNA6', 'PPT1', 
                     'MIB2', 'CDK11B', 'FAAP20',
                     'CD3E','CD3D', 'TRAC', 'TRBC1', 'TRBC2', 
                     'CCN1', 'DPT', 'PDLIM3',
                     'PLA2G2A', 'ACKR1', 'PRELP',
                     'COL8A2', 'PODN', 'PTGER3',
                     'MXRA8', 'MEGF6', 'ID3',
                     'PANK4', 'TNFRSF14', 'PTPRU',
                     'AQP1', 'RAMP3', 'PODXL',
                     'TNFRSF1B', 'DHRS3', 'ALPL', 
                     'HSPB7', 'S100A1', 'ATP1A2',
                     'KIF1B', 'SYNC', 'GADD45A',
                     'SLC2A5', 'PADI2', 'TCEA3',
                     'COL3A1', 'FN1', 'COL6A3', 'ACTB','CD74', 
                     'ENO1', 'MICOS10', 'NBL1',
                     'AGRN', 'CTNNBIP1', 'MFAP2', 
                     'INTS11', 'CCNL2', 'SSU72',
                     'ICMT', 'ATP5IF1', 'MAP7D1',
                     'IGKC','IGLC1', 'IGKV4-1','S100A8','S100A9', 
                     'NADK', 'NPHP4', 'NOL9', 
                     'PRKCZ', 'PER3', 'UBE4B', 
                     'UBE2J2', 'ACAP3', 'VWA1', 
                     'CDC20', 'HIST2H2AB', 'TACC3',
                     'NOC2L', 'B3GALT6', 'AURKAIP1',
                     'IFI44L', 'IFI44', 'GBP5',
                     'HES4', 'CPTP', 'UBR4',
                     'LCE3E', 'LCE3D', 'IVL', 
                     'GRHL3', 'PDZK1IP1', 'CRCT1',
                     'DVL1', 'AJAP1', 'CASZ1', 
                     'PLCH2', 'TPRG1L', 'GPR153'
                     )

DotPlot(Seurat, features = cluster_features) + RotatedAxis() + scale_colour_viridis() +DarkTheme()

# Save the last plot

ggsave(
  "Dot plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 60,
  height = 25,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

## Cell cluster percentage stacked bar plot

In [ ]:
cluster_counts <- as.data.frame(table(Seurat@meta.data$CellType))
colnames(cluster_counts) <- c("Cluster", "CellCount")
cluster_counts$Percentage <- (cluster_counts$CellCount / sum(cluster_counts$CellCount)) * 100

# Define custom colors (change if needed)
custom_colors <- rainbow(length(unique(cluster_counts$Cluster)))

# Create the stacked bar plot with percentages
ggplot(cluster_counts, aes(x = "", y = Percentage, fill = as.factor(Cluster))) +
  geom_bar(stat = "identity", width = 1) +
  scale_fill_manual(values = custom_colors) +  # Use custom colors
  theme_dark() +  # Apply dark theme
  labs(title = "Cluster Distribution (Percentage)", y = "Percentage of Cells", fill = "Cluster") +
  theme(
    panel.background = element_rect(fill = "black", color = NA),   # Black background
    plot.background = element_rect(fill = "black", color = NA),    # Black plot area
    panel.grid.major = element_line(color = "gray30"),  # Dark gray grid
    panel.grid.minor = element_blank(),
    axis.text = element_text(color = "white"),   # White axis text
    axis.title = element_text(color = "white"),  # White axis title
    legend.background = element_rect(fill = "black"),  # Dark legend
    legend.text = element_text(color = "white"),
    legend.title = element_text(color = "white")
  ) +
  scale_y_continuous(labels = scales::percent_format(scale = 1))  # Show percentages on y-axis

ggsave(
  "Cell Stacked Bar Plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 6,
  height = 40,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

# Neighbourhood analysis

In [ ]:
Seurat_FOV@meta.data

In [ ]:
# Calculate cell cluster distances

library(ComplexHeatmap)
library(circlize)

# Extract spatial coordinates and CellType
df <- Seurat@meta.data %>%
  select(center_x, center_y, CellType)

# Define chunk size
chunk_size <- 1000  # Adjust based on available RAM
num_chunks <- ceiling(nrow(df) / chunk_size)

# Create an empty distance matrix for CellTypes
celltypes <- unique(df$CellType)
celltype_dist <- matrix(NA, length(celltypes), length(celltypes), dimnames = list(celltypes, celltypes))

# Function to compute pairwise distances for a chunk
compute_chunk_distances <- function(chunk) {
  dist_mat <- as.matrix(dist(chunk[, c("center_x", "center_y")]))  # Euclidean distance
  return(dist_mat)
}

# Process in chunks
for (i in seq_len(num_chunks)) {
  cat("Processing chunk", i, "of", num_chunks, "\n")
  
  # Get chunk of data
  chunk <- df[((i - 1) * chunk_size + 1):min(i * chunk_size, nrow(df)), ]
  
  # Compute pairwise distances
  dist_chunk <- compute_chunk_distances(chunk)
  
  # Update the CellType distance matrix (average per CellType)
  for (ct1 in unique(chunk$CellType)) {
    for (ct2 in unique(chunk$CellType)) {
      if (ct1 != ct2) {
        dists <- dist_chunk[chunk$CellType == ct1, chunk$CellType == ct2]
        celltype_dist[ct1, ct2] <- mean(dists, na.rm = TRUE)  # Average distance
      }
    }
  }
}

# Convert distance to similarity (1 - normalized distance)
celltype_similarity <- 1 - (celltype_dist / max(celltype_dist, na.rm = TRUE))

In [ ]:
# Create a heatmap with hierarchical clustering

col_fun <- colorRamp2(c(0, 0.5, 1), c("blue", "white", "red"))

jpeg("CellType Neighbourhood Heatmap.jpeg", width = 4750, height = 4000, res = 600)

Heatmap(as.matrix(celltype_similarity),
        name = "Co-localization",
        col = col_fun,  # Apply the corrected colormap
        # cluster_rows = TRUE, 
        # cluster_columns = TRUE,
        show_heatmap_legend = TRUE)  # Ensure legend is displayed

dev.off()

In [ ]:
# Subset for co-localized cell types
Seurat_co_localized <- subset(Seurat, subset = CellType %in% c('15','16','26','24','11','10','4','12','27','21','29','17'))
Seurat_co_localized

In [ ]:
Seurat_co_localized@meta.data$center_x <- as.numeric(Seurat_co_localized@meta.data$center_x)
Seurat_co_localized@meta.data$center_y <- as.numeric(Seurat_co_localized@meta.data$center_y)

# Plot clusters with black background
ggplot(Seurat_co_localized@meta.data, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 2) +
  scale_color_manual(values = rainbow(length(unique(Seurat_co_localized$CellType)))) + # Custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Clusters", color = "CellType")

ggsave(
  "Spatial Co-localized Clusters' Plot.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 41,
  height = 40,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

# Cell-Cell Communication Analysis of the co-localized cell clusters

In [ ]:
library(CellChat)
library(patchwork)
library(ggalluvial)
library(igraph)

In [ ]:
CellChat <- createCellChat(Seurat_co_localized,
#                               group.by = 'Sample',
                              # assay = 'SCT',
                              do.sparse = T)

CellChatDB <- CellChatDB.human

CellChatDB.use <- CellChatDB
CellChat@DB <- CellChatDB.use
CellChat <- subsetData(CellChat)
CellChat <- identifyOverExpressedGenes(CellChat)
CellChat <- identifyOverExpressedInteractions(CellChat)

CellChat <- projectData(CellChat, PPI.human)

CellChat <- computeCommunProb(CellChat)

CellChat <- computeCommunProbPathway(CellChat)

CellChat <- aggregateNet(CellChat)

library(NMF)
library(ggalluvial)

In [ ]:
CellChat@netP$pathways

In [ ]:
selectK(CellChat, pattern = "outgoing")

In [ ]:
selectK(CellChat, pattern = "incoming")

In [ ]:
nPatterns = 3
CellChat <- identifyCommunicationPatterns(CellChat, 
                                          pattern = "outgoing", 
                                          k = nPatterns)
nPatterns = 2
CellChat <- identifyCommunicationPatterns(CellChat, 
                                          pattern = "incoming", 
                                          k = nPatterns)

In [ ]:
# river plot
netAnalysis_river(CellChat, pattern = "outgoing")

ggsave(
  "River Plot Outgoing.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 20,
  height = 20,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
# river plot
netAnalysis_river(CellChat, pattern = "incoming")

ggsave(
  "River Plot Incoming.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 20,
  height = 20,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
# Dot plot of outgoing communications
netAnalysis_dot(CellChat, pattern = "outgoing")

ggsave(
  "Dot Plot Outgoing.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 22,
  height = 14,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
# Dot plot of incoming communications
netAnalysis_dot(CellChat, pattern = "incoming")

ggsave(
  "Dot Plot Incoming.jpeg",
  plot = last_plot(),
  device = "jpeg",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 22,
  height = 14,
  units = "cm",
  dpi = 600,
  limitsize = TRUE,
#   bg = NULL,
)